# EAS 503 Major Project

## Performing Analysis on 311 Requests of San Francisco

## Group 19 

- Sai Vineeth Grandhi 
- Pavan Chandra Sanda
- Diwakar Nambi
- Hruthika Kancherla 

## Introduction

SF311 is a primary customer service center for the City of San Francisco, where the citizens can report the issues like abandoned vehicles, noise complaints, and graffiti etc., Depending on your service request, the team will answer the question or send municipal services to the location to fix the problem. As there will be many requests generated every minute it would be difficult for tracking the request. For this purpose, we have chosen this data set to understand what type of requests are being generated and analyze the service request by using different analyzing techniques. By performing these analysis, we would conclude some of the facts related to service requests.

## Data Description 

- This Dataset is taken from official San Francisco website - https://data.sfgov.org/d/vw6y-z8j6?category=City-Infrastructure&view_name=311-Cases 
- The Number of Records in the Dataset - 2.4 Million Rows and 20 Columns.
- The Data has been filtered for 3 years 2019 to 2022.

## Data Base Schema 

![](ERD.jpeg)

In [1]:
# importing required libraries 

import pandas as pd
import sqlite3
from sqlite3 import Error

import matplotlib.pyplot as plt 
import seaborn as sns 
import plotly.express as px
import plotly.graph_objects as go 

In [2]:
# Database required functions 

def create_connection(db_file, delete_db=False):
    import os
    if delete_db and os.path.exists(db_file):
        os.remove(db_file)

    conn = None
    try:
        conn = sqlite3.connect(db_file)
        conn.execute("PRAGMA foreign_keys = 1")
    except Error as e:
        print(e)

    return conn


def create_table(conn, create_table_sql, drop_table_name=None):
    
    if drop_table_name: # You can optionally pass drop_table_name to drop the table. 
        try:
            c = conn.cursor()
            c.execute("""DROP TABLE IF EXISTS %s""" % (drop_table_name))
        except Error as e:
            print(e)
    
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)
        
def execute_sql_statement(sql_statement, conn):
    cur = conn.cursor()
    cur.execute(sql_statement)
    rows = cur.fetchall()
    return rows

## Non Normalized Database

In [3]:
# creating non-normalized.db 

nonnormalized_table = """CREATE TABLE SF311 (
    CaseID INTEGER PRIMARY KEY, 
    Opened TEXT, 
    Closed TEXT, 
    Updated TEXT, 
    Status TEXT, 
    StatusNotes TEXT, 
    ResponsibleAgency TEXT, 
    Category TEXT, 
    RequestType TEXT, 
    RequestDetails TEXT, 
    Address TEXT, 
    Street TEXT, 
    SupervisorDistrict TEXT, 
    Neighborhood TEXT, 
    PoliceDistrict TEXT, 
    Latitude TEXT, 
    Longitude TEXT, 
    Point TEXT, 
    Source TEXT, 
    MediaURL TEXT
);"""

db_name = 'nonnormalized.db'

conn = create_connection(db_name, delete_db= True)
create_table(conn, nonnormalized_table) 

In [4]:
# reading the data from file and creating list of tuples 

case_file = '311_Cases.csv'
import csv
insert_rows = []
with open(case_file, "r") as csvfile:
    reader_variable = csv.reader(csvfile, delimiter=",")

    for row in reader_variable: 
        insert_rows.append(tuple(row))

insert_rows = insert_rows[1:]

In [5]:
# inserting the list of tuples to nonnormalized.db 

insert_st = 'INSERT INTO SF311 VALUES (' + '?,'*19 + '?);'

conn = create_connection(db_name) 
cur = conn.cursor() 
cur.executemany(insert_st, insert_rows) 
conn.commit()

## Normalized Database

In [6]:
# creating tables in normalized db 
# creating Source table

conn = create_connection("nonnormalized.db")
sql_source = "SELECT DISTINCT(Source) AS Source FROM SF311 ORDER BY Source;"
source_rows = execute_sql_statement(sql_source, conn) 

source_table = """CREATE TABLE Source (
    SourceID INTEGER PRIMARY KEY, 
    Source TEXT NOT NULL
);"""

insert_source = 'INSERT INTO Source VALUES (?, ?);' 

source_insert = [(i, j[0]) for i,j in zip(range(101, 101+len(source_rows)), source_rows)]

conn = create_connection("normalized.db")
create_table(conn, source_table, drop_table_name= 'Source')
cur = conn.cursor() 
cur.executemany(insert_source, source_insert) 
conn.commit() 

In [7]:
# creating Neighborhood table

conn = create_connection("nonnormalized.db")
sql_neig = "SELECT DISTINCT(Neighborhood) AS Neighborhood FROM SF311 ORDER BY Neighborhood;"
neighborhood_rows = execute_sql_statement(sql_neig, conn)

neighborhood_table = """CREATE TABLE Neighborhood (
    NeighborhoodID INTEGER PRIMARY KEY, 
    Neighborhood TEXT NOT NULL
);"""

insert_neig = 'INSERT INTO Neighborhood VALUES (?, ?);' 

neig_insert = [(i, j[0]) for i,j in zip(range(201, 201+len(neighborhood_rows)), neighborhood_rows)]

conn = create_connection("normalized.db")
create_table(conn, neighborhood_table, drop_table_name= 'Neighborhood')
cur = conn.cursor() 
cur.executemany(insert_neig, neig_insert) 
conn.commit()

In [8]:
# creating Location table

conn = create_connection("nonnormalized.db")
sql_location = "SELECT DISTINCT Latitude, Longitude, Point FROM SF311;"
location_rows = execute_sql_statement(sql_location, conn)

location_table = """CREATE TABLE Location (
    LocationID INTEGER PRIMARY KEY, 
    Latitude INTEGER NOT NULL,
    Longitude INTEGER NOT NULL, 
    Point TEXT
);"""

insert_loc = 'INSERT INTO Location VALUES (?, ?, ?, ?);' 

loc_insert = [(i, j[0], j[1], j[2]) for i,j in zip(range(1000000, 1000000+len(location_rows)), location_rows)]
conn = create_connection("normalized.db")
create_table(conn, location_table, drop_table_name= 'Location')
cur = conn.cursor() 
cur.executemany(insert_loc, loc_insert) 
conn.commit()

In [9]:
# creating Category table

conn = create_connection("nonnormalized.db")
sql_cat = "SELECT DISTINCT(Category) AS Category FROM SF311 ORDER BY Category;"
cat_rows = execute_sql_statement(sql_cat, conn)

cat_table = """CREATE TABLE Category (
    CategoryID INTEGER PRIMARY KEY, 
    Category TEXT NOT NULL
);"""

insert_cat = 'INSERT INTO Category VALUES (?, ?);' 

cat_insert = [(i, j[0]) for i,j in zip(range(301, 301+len(cat_rows)), cat_rows)]

conn = create_connection("normalized.db")
create_table(conn, cat_table, drop_table_name= 'Category')
cur = conn.cursor() 
cur.executemany(insert_cat, cat_insert) 
conn.commit()

In [10]:
# creating SupervisorDistrict table

conn = create_connection("nonnormalized.db")
sql_supr = "SELECT DISTINCT(SupervisorDistrict) AS SupervisorDistrict FROM SF311 ORDER BY SupervisorDistrict;"
supr_rows = execute_sql_statement(sql_supr, conn)

supr_table = """CREATE TABLE SupervisorDistrict (
    SupervisorDistrictID INTEGER PRIMARY KEY, 
    SupervisorDistrict TEXT NOT NULL
);"""

insert_supr = 'INSERT INTO SupervisorDistrict VALUES (?, ?);' 

supr_insert = [(i, j[0]) for i,j in zip(range(1000, 1000+len(supr_rows)), supr_rows)]

conn = create_connection("normalized.db")
create_table(conn, supr_table, drop_table_name= 'SupervisorDistrict')
cur = conn.cursor() 
cur.executemany(insert_supr, supr_insert) 
conn.commit()

In [11]:
# creating PoliceDistrict table

conn = create_connection("nonnormalized.db")
sql_pol = "SELECT DISTINCT(PoliceDistrict) AS PoliceDistrict FROM SF311 ORDER BY PoliceDistrict;"
pol_rows = execute_sql_statement(sql_pol, conn)

pol_table = """CREATE TABLE PoliceDistrict (
    PoliceDistrictID INTEGER PRIMARY KEY, 
    PoliceDistrict TEXT NOT NULL
);"""

insert_pol = 'INSERT INTO PoliceDistrict VALUES (?, ?);' 

pol_insert = [(i, j[0]) for i,j in zip(range(2000, 2000+len(pol_rows)), pol_rows)]

conn = create_connection("normalized.db")
create_table(conn, pol_table, drop_table_name= 'PoliceDistrict')
cur = conn.cursor() 
cur.executemany(insert_pol, pol_insert) 
conn.commit()

In [12]:
# creating ResponsibleAgency table

conn = create_connection("nonnormalized.db")
sql_res = "SELECT DISTINCT(ResponsibleAgency) AS ResponsibleAgency FROM SF311 ORDER BY ResponsibleAgency;"
res_rows = execute_sql_statement(sql_res, conn)

res_table = """CREATE TABLE ResponsibleAgency (
    ResponsibleAgencyID INTEGER PRIMARY KEY, 
    ResponsibleAgency TEXT NOT NULL
);"""

insert_res = 'INSERT INTO ResponsibleAgency VALUES (?, ?);' 

res_insert = [(i, j[0]) for i,j in zip(range(3000, 3000+len(res_rows)), res_rows)]

conn = create_connection("normalized.db")
create_table(conn, res_table, drop_table_name= 'ResponsibleAgency')
cur = conn.cursor() 
cur.executemany(insert_res, res_insert) 
conn.commit()

In [13]:
# Creating Case Table 

conn = create_connection("nonnormalized.db")
sql_sf = "SELECT * FROM SF311;" 
all_rows = execute_sql_statement(sql_sf, conn)

sf_table = """CREATE TABLE SF311 (
    CaseID INTEGER PRIMARY KEY, 
    Opened TEXT, 
    Closed TEXT, 
    Status TEXT, 
    Address TEXT, 
    NeighborhoodID INTEGER NOT NULL, 
    SourceID INTEGER NOT NULL, 
    CategoryID INTEGER NOT NULL, 
    LocationID INTEGER NOT NULL,
    SupervisorDistrictID INTERGER NOT NULL, 
    PoliceDistrictID INTEGER NOT NULL, 
    ResponsibleAgencyID INTEGER NOT NULL, 
    FOREIGN KEY (NeighborhoodID) REFERENCES Neighborhood(NeighborhoodID),
    FOREIGN KEY (SourceID) REFERENCES Source(SourceID),
    FOREIGN KEY (CategoryID) REFERENCES Category(CategoryID),
    FOREIGN KEY (LocationID) REFERENCES Location(LocationID),
    FOREIGN KEY (SupervisorDistrictID) REFERENCES SupervisorDistrict(SupervisorDistrictID),
    FOREIGN KEY (PoliceDistrictID) REFERENCES PoliceDistrict(PoliceDistrictID),
    FOREIGN KEY (ResponsibleAgencyID) REFERENCES ResponsibleAgency(ResponsibleAgencyID)
);"""

conn = create_connection('normalized.db')
neighborhood = execute_sql_statement('SELECT * FROM Neighborhood;', conn)
source = execute_sql_statement('SELECT * FROM Source;', conn)
category = execute_sql_statement('SELECT * FROM Category;', conn)
location = execute_sql_statement('SELECT * FROM Location;', conn)
supervisor = execute_sql_statement('SELECT * FROM SupervisorDistrict;', conn)
police = execute_sql_statement('SELECT * FROM PoliceDistrict;', conn)
responsibleagency = execute_sql_statement('SELECT * FROM ResponsibleAgency;', conn)

neighborhood_dict = {}
source_dict = {}
category_dict = {} 
location_dict = {} 
supervisor_dict = {} 
police_dict = {} 
responsibleagency_dict = {} 

for row in neighborhood: 
    neighborhood_dict.update({row[1]: row[0]})

for row in source: 
    source_dict.update({row[1]: row[0]})

for row in category: 
    category_dict.update({row[1]: row[0]})

for row in location: 
    location_dict.update({row[3]: row[0]})

for row in supervisor: 
    supervisor_dict.update({row[1]: row[0]})

for row in police: 
    police_dict.update({row[1]: row[0]})

for row in responsibleagency: 
    responsibleagency_dict.update({row[1]: row[0]}) 

sf_insert = []

for i in all_rows: 
    sf_insert.append((i[0], i[1], i[2], i[4], i[10], 
    neighborhood_dict[i[13]], source_dict[i[18]], category_dict[i[7]], 
    location_dict[i[17]], supervisor_dict[i[12]], police_dict[i[14]], 
    responsibleagency_dict[i[6]]))

insert_sf = 'INSERT INTO SF311 VALUES (' + '?, '*11 +'?);'

conn = create_connection("normalized.db")
create_table(conn, sf_table, drop_table_name= 'SF311')
cur = conn.cursor() 
cur.executemany(insert_sf, sf_insert) 
conn.commit()

## Data Analysis

### No of Requests

In [14]:
# setting connection to normalized.db 

normalized_file = "normalized.db"
conn = create_connection(normalized_file)

In [15]:
# latitude and longitude density 

df = pd.read_sql_query("""SELECT strftime('%Y', DATE(SUBSTR(s.Opened, 7, 4) ||'-'||SUBSTR(s.Opened, 1, 2) ||'-'||SUBSTR(s.Opened, 4,2))) AS Year, l.Latitude, l.Longitude, Count(*) AS Requests
                        FROM SF311 s
                        JOIN Location l 
                        ON l.LocationID = s.LocationID 
                        GROUP BY Year, l.Latitude, l.Longitude
                        HAVING Requests > 10
                        ORDER BY Requests DESC; """, conn)

df.head()

,Year,Latitude,Longitude,Requests
0,2019,0.0000,0.00,17379
1,2020,0.0000,0.00,11817
2,2021,0.0000,0.00,8187
3,2022,0.0000,0.00,7608
4,2020,37.7797,-122.42,1444


In [16]:
fig = px.density_mapbox(df, lat= 'Latitude', lon = 'Longitude', z= 'Requests', zoom= 10, animation_frame= 'Year')
fig.update_layout(mapbox_style="open-street-map", title = 'Number of Requests In San Fransico')
fig.show() 

In [23]:
# No of requests YoY Trend 

df = pd.read_sql_query("""SELECT strftime('%Y', DATE(SUBSTR(Opened, 7, 4) ||'-'||SUBSTR(Opened, 1, 2) ||'-'||SUBSTR(Opened, 4,2))) AS Year, Count(*) AS RequestCount
                  FROM SF311 
                  GROUP BY Year 
                  ORDER BY Year ASC;""", conn)

df.head()

,Year,RequestCount
0,2019,657082
1,2020,629556
2,2021,633806
3,2022,561031


In [26]:
fig = px.line(df, x = 'Year', y= 'RequestCount', title= 'Number of 311 Requests in San Francisco', markers= True) 
fig.update_layout(width = 1200, height = 600, xaxis_tickangle=-45)
fig.show()

### Analysis Of Requests Based On Neighborhood

In [105]:
# No of requests by Different Neighborhood 

df = pd.read_sql_query("""SELECT n.Neighborhood, COUNT(*) AS Count 
                    FROM SF311 sf 
                    JOIN Neighborhood n
                    ON sf.NeighborhoodID = n.NeighborhoodID
                    GROUP BY n.Neighborhood
                    HAVING n.Neighborhood != ''
                    ORDER BY Count DESC
                    LIMIT 10;""", conn)

df.head()

,Neighborhood,Count
0,Mission,265046
1,South of Market,172682
2,Outer Sunset,78753
3,Tenderloin,72339
4,Outer Richmond,68900


In [109]:
fig = px.pie(df, values= 'Count', names= 'Neighborhood', color = 'Count', title = 'Number of Requests by Neighborhood', hole = .5) 
fig.update_layout(width = 800, height = 400)
fig.show()

### Analysis Of Requests Based On Source

In [112]:
# No of requests by Different Sources 

df = pd.read_sql_query("""SELECT s.Source, COUNT(*) AS Count 
                    FROM SF311 sf 
                    JOIN Source s 
                    ON sf.SourceID = s.SourceID
                    GROUP BY s.Source
                    ORDER BY Count DESC;""", conn)

df.head()

,Source,Count
0,Mobile/Open311,1494312
1,Phone,607732
2,Web,327785
3,Integrated Agency,40767
4,Twitter,10620


In [113]:
fig = px.pie(df, values= 'Count', names= 'Source', color = 'Count', title = 'Number of Requests by Source', hole = .5) 
fig.update_layout(width = 800, height = 400)
fig.show()

In [44]:
# YoY Trend on Requests by Source 

df = pd.read_sql_query("""SELECT strftime('%Y', DATE(SUBSTR(sf.Opened, 7, 4) ||'-'||SUBSTR(sf.Opened, 1, 2) ||'-'||SUBSTR(sf.Opened, 4,2))) AS Year, s.Source, COUNT(*) AS Count 
                    FROM SF311 sf 
                    JOIN Source s 
                    ON sf.SourceID = s.SourceID
                    GROUP BY Year, s.Source
                    ORDER BY Year, Count DESC;""", conn) 

df.head()

,Year,Source,Count
0,2019,Mobile/Open311,370177
1,2019,Phone,184657
2,2019,Web,84426
3,2019,Integrated Agency,14442
4,2019,Twitter,3196


In [43]:
fig = px.line(df, x= 'Year', y = 'Count', color = 'Source', markers= True,  title= 'YoY Trend of Requests By Source')
fig.update_layout(width = 1000, height = 500, xaxis_tickangle=-45)
fig.show()

### Analysis Of Requests Based on Categories

In [122]:
# No of requests by Different Categories 

df = pd.read_sql_query("""SELECT c.Category, COUNT(*) AS Count 
                    FROM SF311 sf 
                    JOIN Category c
                    ON sf.CategoryID = c.CategoryID
                    GROUP BY c.Category
                    ORDER BY Count DESC 
                    LIMIT 30;""", conn)

df.head()

,Category,Count
0,Street and Sidewalk Cleaning,1053882
1,Parking Enforcement,285498
2,Graffiti,252216
3,Encampments,166714
4,General Request - PUBLIC WORKS,97471


In [123]:
fig = px.bar(df, x= 'Category', y = 'Count', color = 'Count', title = 'Number of Requests by Category') 
fig.update_layout(width = 1400, height = 800, xaxis_tickangle=-90)
fig.show()

In [89]:
# No of Opened and Closed Cases for Categories

df = pd.read_sql_query("""SELECT strftime('%Y', DATE(SUBSTR(s.Opened, 7, 4) ||'-'||SUBSTR(s.Opened, 1, 2) ||'-'||SUBSTR(s.Opened, 4,2))) AS Year, c.Category, s.Status, Count(*) AS Number 
                FROM SF311 s 
                JOIN Category c
                ON s.CategoryID = c.CategoryID
                GROUP BY Year, c.Category, s.Status
                HAVING c.Category != '' AND Number > 1000
                ORDER BY Year, Number DESC; """, conn)

df.head()

,Year,Category,Status,Number
0,2019,Street and Sidewalk Cleaning,Closed,247647
1,2019,Parking Enforcement,Closed,78138
2,2019,Encampments,Closed,69691
3,2019,Graffiti,Closed,56999
4,2019,Abandoned Vehicle,Closed,30127


In [90]:
fig = px.bar(df, x = 'Year', y = 'Number', color= 'Status', barmode= 'group', animation_frame= 'Category', text_auto='.2s', title = 'Number of Open & Closed Cases YoY By Categories')
fig.update_layout(width = 1200, height = 600, xaxis_tickangle=-45)
fig.show()

In [98]:
# Average ResponseTime YoY for Categories

df = pd.read_sql_query("""SELECT strftime('%Y', DATE(SUBSTR(s.Opened, 7, 4) ||'-'||SUBSTR(s.Opened, 1, 2) ||'-'||SUBSTR(s.Opened, 4,2))) AS Year, c.Category, 
                    ROUND(AVG(JULIANDAY(DATE(SUBSTR(s.Closed, 7, 4) ||'-'||SUBSTR(s.Closed, 1, 2) ||'-'||SUBSTR(s.Closed, 4,2))) - JULIANDAY(DATE(SUBSTR(s.Opened, 7, 4) ||'-'||SUBSTR(s.Opened, 1, 2) ||'-'||SUBSTR(s.Opened, 4,2))))) AS AvgResolutionTime 
                    FROM SF311 s 
                    JOIN Category c
                    ON c.CategoryID = s.CategoryID
                    WHERE s.Closed != ''
                    GROUP BY Year, c.Category 
                    HAVING c.Category != '' AND AvgResolutionTime > 200
                    ORDER BY AvgResolutionTime DESC;
                    """, conn)

df.head()

,Year,Category,AvgResolutionTime
0,2019,General Request - CONTRACT ADMINISTRATION,1081.0
1,2019,General Request -,784.0
2,2020,General Request - CONTRACT ADMINISTRATION,726.0
3,2019,General Request - CITY ATTORNEY,583.0
4,2019,General Request - ELECTIONS,551.0


In [99]:
fig = px.line(df, x = 'Year', y= 'AvgResolutionTime', animation_frame= 'Category', title= 'Average Resolution Time on YoY Requests By Category', markers= True) 
fig.update_layout(width = 1200, height = 600, xaxis_tickangle=-45)
fig.show()

### Analysis Of Requests Based on Police District

In [114]:
# No of requests by Police District

df = pd.read_sql_query("""SELECT p.PoliceDistrict, COUNT(*) AS Count 
                    FROM SF311 sf 
                    JOIN PoliceDistrict p
                    ON sf.PoliceDistrictID = p.PoliceDistrictID
                    GROUP BY p.PoliceDistrict
                    HAVING p.PoliceDistrict != ''
                    ORDER BY Count DESC;""", conn)

df.head()

,PoliceDistrict,Count
0,MISSION,464139
1,NORTHERN,326038
2,INGLESIDE,233754
3,CENTRAL,233317
4,BAYVIEW,228189


In [115]:
fig = px.pie(df, values= 'Count', names= 'PoliceDistrict', color = 'Count', title = 'Number of Requests by Police District', hole = .5) 
fig.update_layout(width = 800, height = 400)
fig.show()

In [53]:
# YoY Trend on Requests by Police District 

df = pd.read_sql_query("""SELECT strftime('%Y', DATE(SUBSTR(sf.Opened, 7, 4) ||'-'||SUBSTR(sf.Opened, 1, 2) ||'-'||SUBSTR(sf.Opened, 4,2))) AS Year, p.PoliceDistrict, COUNT(*) AS Count 
                    FROM SF311 sf 
                    JOIN PoliceDistrict p
                    ON sf.PoliceDistrictID = p.PoliceDistrictID
                    GROUP BY Year, p.PoliceDistrict
                    HAVING p.PoliceDistrict != ''
                    ORDER BY Year, Count DESC;""", conn) 

df.head()

,Year,PoliceDistrict,Count
0,2019,MISSION,125852
1,2019,NORTHERN,78181
2,2019,SOUTHERN,70969
3,2019,INGLESIDE,68615
4,2019,TARAVAL,59930


In [54]:
fig = px.line(df, x= 'Year', y = 'Count', color = 'PoliceDistrict', markers= True,  title= 'YoY Trend of Requests By Police District')
fig.update_layout(width = 1000, height = 500, xaxis_tickangle=-45)
fig.show()

In [60]:
# No of Opened and Closed Cases for Police District 

df = pd.read_sql_query("""SELECT strftime('%Y', DATE(SUBSTR(s.Opened, 7, 4) ||'-'||SUBSTR(s.Opened, 1, 2) ||'-'||SUBSTR(s.Opened, 4,2))) AS Year, p.PoliceDistrict, s.Status, Count(*) AS Number 
                FROM SF311 s 
                JOIN PoliceDistrict p 
                ON p.PoliceDistrictID = s.PoliceDistrictID 
                GROUP BY Year, p.PoliceDistrict, s.Status
                HAVING p.PoliceDistrict != ''
                ORDER BY Year, Number DESC; """, conn)

df.head()

,Year,PoliceDistrict,Status,Number
0,2019,MISSION,Closed,125468
1,2019,NORTHERN,Closed,77940
2,2019,SOUTHERN,Closed,70696
3,2019,INGLESIDE,Closed,68195
4,2019,TARAVAL,Closed,59640


In [62]:
fig = px.bar(df, x = 'Year', y = 'Number', color= 'Status', barmode= 'group', animation_frame= 'PoliceDistrict', text_auto='.2s', title = 'Number of Open & Closed Cases YoY By Police Districts')
fig.update_layout(width = 1200, height = 500, xaxis_tickangle=-45)
fig.show()

In [63]:
# No of Requests YoY Based on Months By Police District


df = pd.read_sql_query("""SELECT strftime('%Y', DATE(SUBSTR(s.Opened, 7, 4) ||'-'||SUBSTR(s.Opened, 1, 2) ||'-'||SUBSTR(s.Opened, 4,2))) AS Year, 
                    CASE
                        WHEN strftime('%m', DATE(SUBSTR(s.Opened, 7, 4) ||'-'||SUBSTR(s.Opened, 1, 2) ||'-'||SUBSTR(s.Opened, 4,2))) = '01' THEN 'January'
                        WHEN strftime('%m', DATE(SUBSTR(s.Opened, 7, 4) ||'-'||SUBSTR(s.Opened, 1, 2) ||'-'||SUBSTR(s.Opened, 4,2))) = '02' THEN 'February'
                        WHEN strftime('%m', DATE(SUBSTR(s.Opened, 7, 4) ||'-'||SUBSTR(s.Opened, 1, 2) ||'-'||SUBSTR(s.Opened, 4,2))) = '03' THEN 'March'
                        WHEN strftime('%m', DATE(SUBSTR(s.Opened, 7, 4) ||'-'||SUBSTR(s.Opened, 1, 2) ||'-'||SUBSTR(s.Opened, 4,2))) = '04' THEN 'April'
                        WHEN strftime('%m', DATE(SUBSTR(s.Opened, 7, 4) ||'-'||SUBSTR(s.Opened, 1, 2) ||'-'||SUBSTR(s.Opened, 4,2))) = '05' THEN 'May'
                        WHEN strftime('%m', DATE(SUBSTR(s.Opened, 7, 4) ||'-'||SUBSTR(s.Opened, 1, 2) ||'-'||SUBSTR(s.Opened, 4,2))) = '06' THEN 'June'
                        WHEN strftime('%m', DATE(SUBSTR(s.Opened, 7, 4) ||'-'||SUBSTR(s.Opened, 1, 2) ||'-'||SUBSTR(s.Opened, 4,2))) = '07' THEN 'July'
                        WHEN strftime('%m', DATE(SUBSTR(s.Opened, 7, 4) ||'-'||SUBSTR(s.Opened, 1, 2) ||'-'||SUBSTR(s.Opened, 4,2))) = '08' THEN 'August'
                        WHEN strftime('%m', DATE(SUBSTR(s.Opened, 7, 4) ||'-'||SUBSTR(s.Opened, 1, 2) ||'-'||SUBSTR(s.Opened, 4,2))) = '09' THEN 'September'
                        WHEN strftime('%m', DATE(SUBSTR(s.Opened, 7, 4) ||'-'||SUBSTR(s.Opened, 1, 2) ||'-'||SUBSTR(s.Opened, 4,2))) = '10' THEN 'October'
                        WHEN strftime('%m', DATE(SUBSTR(s.Opened, 7, 4) ||'-'||SUBSTR(s.Opened, 1, 2) ||'-'||SUBSTR(s.Opened, 4,2))) = '11' THEN 'November'
                        WHEN strftime('%m', DATE(SUBSTR(s.Opened, 7, 4) ||'-'||SUBSTR(s.Opened, 1, 2) ||'-'||SUBSTR(s.Opened, 4,2))) = '12' THEN 'December'
                    END Month, p.PoliceDistrict, COUNT(*) AS RequestCount 
                    FROM SF311 s
                    JOIN PoliceDistrict p 
                    ON p.PoliceDistrictID = s.PoliceDistrictID
                    GROUP BY Year, Month, p.PoliceDistrict
                    HAVING p.PoliceDistrict != ''
                    ORDER BY Month;""", conn)

df.head()

,Year,Month,PoliceDistrict,RequestCount
0,2019,April,BAYVIEW,4759
1,2019,April,CENTRAL,4878
2,2019,April,INGLESIDE,5511
3,2019,April,MISSION,10343
4,2019,April,NORTHERN,5987


In [65]:
fig = px.bar(df, x='RequestCount', y='PoliceDistrict', text_auto='.2s',
            title="Number of Requests Year On Year By Police District", animation_frame= 'Year', color='Month')
fig.update_layout(width = 1200, height = 600, xaxis_tickangle=-45)
fig.show()

In [66]:
# Average ResponseTime YoY for Police District

df = pd.read_sql_query("""SELECT strftime('%Y', DATE(SUBSTR(s.Opened, 7, 4) ||'-'||SUBSTR(s.Opened, 1, 2) ||'-'||SUBSTR(s.Opened, 4,2))) AS Year, p.PoliceDistrict, 
                    ROUND(AVG(JULIANDAY(DATE(SUBSTR(s.Closed, 7, 4) ||'-'||SUBSTR(s.Closed, 1, 2) ||'-'||SUBSTR(s.Closed, 4,2))) - JULIANDAY(DATE(SUBSTR(s.Opened, 7, 4) ||'-'||SUBSTR(s.Opened, 1, 2) ||'-'||SUBSTR(s.Opened, 4,2))))) AS AvgResolutionTime 
                    FROM SF311 s 
                    JOIN PoliceDistrict p 
                    ON p.PoliceDistrictID = s.PoliceDistrictID 
                    WHERE s.Closed != ''
                    GROUP BY Year, p.PoliceDistrict 
                    HAVING p.PoliceDistrict != ''
                    ORDER BY AvgResolutionTime DESC;
                    """, conn)

df.head()

,Year,PoliceDistrict,AvgResolutionTime
0,2019,TARAVAL,20.0
1,2019,CENTRAL,18.0
2,2019,INGLESIDE,18.0
3,2019,PARK,18.0
4,2019,BAYVIEW,17.0


In [69]:
fig = px.line(df, x = 'Year', y= 'AvgResolutionTime', animation_frame= 'PoliceDistrict', title= 'Average Resolution Time on YoY Requests By Police District', markers= True) 
fig.update_layout(width = 1200, height = 600, xaxis_tickangle=-45)
fig.show()

### Analysis Of Requests Based on Responsible Agency

In [120]:
# No of requests by Responsible Agency 

df = pd.read_sql_query("""SELECT r.ResponsibleAgency, COUNT(*) AS Count 
                    FROM SF311 sf 
                    JOIN ResponsibleAgency r
                    ON sf.ResponsibleAgencyID = r.ResponsibleAgencyID
                    GROUP BY r.ResponsibleAgency
                    HAVING r.ResponsibleAgency != ''
                    ORDER BY Count DESC
                    LIMIT 30;""", conn)

df.head()

,ResponsibleAgency,Count
0,DPW Ops Queue,939805
1,Recology_Abandoned,423806
2,Parking Enforcement Dispatch Queue,235964
3,Recology_Overflowing,68250
4,HSOC Queue,65536


In [121]:
fig = px.bar(df, x= 'ResponsibleAgency', y = 'Count', color = 'Count', title = 'Number of Requests by ResponsibleAgency') 
fig.update_layout(width = 1400, height = 800, xaxis_tickangle=-90)
fig.show()

In [124]:
# Average ResponseTime YoY for Responsible Agency

df = pd.read_sql_query("""SELECT strftime('%Y', DATE(SUBSTR(s.Opened, 7, 4) ||'-'||SUBSTR(s.Opened, 1, 2) ||'-'||SUBSTR(s.Opened, 4,2))) AS Year, r.ResponsibleAgency, 
                    ROUND(AVG(JULIANDAY(DATE(SUBSTR(s.Closed, 7, 4) ||'-'||SUBSTR(s.Closed, 1, 2) ||'-'||SUBSTR(s.Closed, 4,2))) - JULIANDAY(DATE(SUBSTR(s.Opened, 7, 4) ||'-'||SUBSTR(s.Opened, 1, 2) ||'-'||SUBSTR(s.Opened, 4,2))))) AS AvgResolutionTime 
                    FROM SF311 s 
                    JOIN ResponsibleAgency r 
                    ON r.ResponsibleAgencyID = s.ResponsibleAgencyID 
                    WHERE s.Closed != ''
                    GROUP BY Year, r.ResponsibleAgency 
                    HAVING r.ResponsibleAgency != ''
                    ORDER BY AvgResolutionTime DESC
                    LIMIT 30;
                    """, conn)

df.head()

,Year,ResponsibleAgency,AvgResolutionTime
0,2019,Contract Administration - G,1159.0
1,2020,City College - G,886.0
2,2019,DPH - Other - G,784.0
3,2020,Contract Administration - G,756.0
4,2019,DPH - Community Health Network - G,709.0


In [125]:
fig = px.line(df, x = 'Year', y= 'AvgResolutionTime', animation_frame= 'ResponsibleAgency', title= 'Average Resolution Time on YoY Requests By Responsible Agency', markers= True) 
fig.update_layout(width = 1200, height = 600, xaxis_tickangle=-45)
fig.show()